# Using transformers for DisjointTimeBasedCesnetDataset

### Import

In [1]:
import numpy as np
import logging

from cesnet_tszoo.utils.enums import AgreggationType, SourceType, TransformerType, DatasetType
from cesnet_tszoo.datasets import CESNET_TimeSeries24
from cesnet_tszoo.configs import DisjointTimeBasedConfig # Disjoint dataset MUST use DisjointTimeBasedConfig

from cesnet_tszoo.utils.transformer import Transformer # For creating custom Transformer

### Setting logger

In [2]:
logging.basicConfig(
    level=logging.INFO,
    format="[%(asctime)s][%(name)s][%(levelname)s] - %(message)s")

### Preparing dataset

In [3]:
disjoint_dataset = CESNET_TimeSeries24.get_dataset(data_root="/some_directory/", source_type=SourceType.IP_ADDRESSES_SAMPLE, aggregation=AgreggationType.AGG_10_MINUTES, dataset_type=DatasetType.DISJOINT_TIME_BASED, display_details=True)

[2025-08-26 09:15:26,167][wrapper_dataset][INFO] - Dataset is disjoint_time_based. Use cesnet_tszoo.configs.DisjointTimeBasedConfig



Dataset details:

    AgreggationType.AGG_10_MINUTES
        Time indices: range(0, 40297)
        Datetime: (datetime.datetime(2023, 10, 9, 0, 3, 49, tzinfo=datetime.timezone.utc), datetime.datetime(2024, 7, 14, 21, 50, 52, tzinfo=datetime.timezone.utc))

    SourceType.IP_ADDRESSES_SAMPLE
        Time series indices: [ 11  20 101 103 118 ... 2003134 2008461 2011839 2022235 2044888], Length=1000; use 'get_available_ts_indices' for full list
        Features with default values: {'n_flows': 0, 'n_packets': 0, 'n_bytes': 0, 'n_dest_ip': 0, 'n_dest_asn': 0, 'n_dest_ports': 0, 'tcp_udp_ratio_packets': 0.5, 'tcp_udp_ratio_bytes': 0.5, 'dir_ratio_packets': 0.5, 'dir_ratio_bytes': 0.5, 'avg_duration': 0, 'avg_ttl': 0}
        
        Additional data: ['ids_relationship', 'weekends_and_holidays']
        


### Transformers

- Transformers are implemented as class.
    - You can create your own or use built-in one.
- Transformer is applied after `default_values` and fillers took care of missing values.
- One transformer is used for all time series.
- Transformer must implement `transform`.
- Transformer must implement `partial_fit` (unless transformer is already fitted and `partial_fit_initialized_transformers` is False).
- To use transformer, train set must be implemented (unless transformer is already fitted and `partial_fit_initialized_transformers` is False).
- You can change used transformer later with `update_dataset_config_and_initialize` or `apply_transformer`.

#### Built-in

In [4]:
# Options

## Supported
TransformerType.STANDARD_SCALER
TransformerType.L2_NORMALIZER
TransformerType.LOG_TRANSFORMER
TransformerType.MAX_ABS_SCALER
TransformerType.MIN_MAX_SCALER

<TransformerType.MIN_MAX_SCALER: 'min_max_scaler'>

In [5]:
config = DisjointTimeBasedConfig(train_ts=500, val_ts=None, test_ts=None, train_time_period=0.5, features_to_take=["n_flows", "n_packets"],
                           transform_with=TransformerType.MIN_MAX_SCALER, nan_threshold=0.5, random_state=1500)
disjoint_dataset.set_dataset_config_and_initialize(config, display_config_details=True, workers=0)

[2025-08-26 09:15:26,177][disjoint_time_based_config][INFO] - Quick validation succeeded.
[2025-08-26 09:15:26,215][disjoint_time_based_config][INFO] - Finalization and validation completed successfully.
[2025-08-26 09:15:26,220][cesnet_dataset][INFO] - Updating config for train set.
100%|██████████| 500/500 [00:00<00:00, 983.03it/s] 
[2025-08-26 09:15:26,739][cesnet_dataset][INFO] - Config initialized successfully.



Config Details
    Used for database: CESNET-TimeSeries24
    Aggregation: AgreggationType.AGG_10_MINUTES
    Source: SourceType.IP_ADDRESSES_SAMPLE

    Time series
        Train time series IDs: [182151  10158  65072  10196 338309 ... 175742 659213  11188  73422 483796], Length=60
        Val time series IDs: None
        Test time series IDs: None
    Time periods
        Train time periods: range(0, 20149)
        Val time periods: None
        Test time periods: None
    Features
        Taken features: ['n_flows', 'n_packets']
        Default values: [0. 0.]
        Time series ID included: True
        Time included: True    
        Time format: TimeFormat.ID_TIME
    Sliding window
        Sliding window size: None
        Sliding window prediction size: None
        Sliding window step size: 1
    Fillers
        Filler type: None
    Transformers
        Transformer type: min_max_scaler
        Are transformers premade: False
        Are premade transformers partial_fitted:

In [6]:
disjoint_dataset.get_train_df(workers=0).head(10)

,id_ip,id_time,n_flows,n_packets
0,182151.0,0.0,0.000079,2.240487e-07
1,182151.0,1.0,0.000045,1.280278e-07
2,182151.0,2.0,0.000045,1.280278e-07
3,182151.0,3.0,0.000056,1.600348e-07
4,182151.0,4.0,0.000090,2.560557e-07
5,182151.0,5.0,0.000034,9.602087e-08
6,182151.0,6.0,0.000068,1.920417e-07
7,182151.0,7.0,0.000045,1.280278e-07
8,182151.0,8.0,0.000102,3.840835e-07
9,182151.0,9.0,0.000000,0.000000e+00


In [7]:
disjoint_dataset.get_transformers()

Or later with:

In [8]:
disjoint_dataset.update_dataset_config_and_initialize(transform_with=TransformerType.MIN_MAX_SCALER, partial_fit_initialized_transformers="config", workers=0)
# Or
disjoint_dataset.apply_transformer(transform_with=TransformerType.MIN_MAX_SCALER, partial_fit_initialized_transformers="config", workers=0)

[2025-08-26 09:15:26,977][cesnet_dataset][INFO] - Re-initialization is required.
[2025-08-26 09:15:27,014][disjoint_time_based_config][INFO] - Finalization and validation completed successfully.
[2025-08-26 09:15:27,017][cesnet_dataset][INFO] - Updating config for train set.
100%|██████████| 60/60 [00:00<00:00, 393.39it/s]
[2025-08-26 09:15:27,172][cesnet_dataset][INFO] - Config initialized successfully.
[2025-08-26 09:15:27,173][cesnet_dataset][INFO] - Configuration has been changed successfuly.
[2025-08-26 09:15:27,173][cesnet_dataset][INFO] - Re-initialization is required.
[2025-08-26 09:15:27,206][disjoint_time_based_config][INFO] - Finalization and validation completed successfully.
[2025-08-26 09:15:27,209][cesnet_dataset][INFO] - Updating config for train set.
100%|██████████| 60/60 [00:00<00:00, 394.76it/s]
[2025-08-26 09:15:27,364][cesnet_dataset][INFO] - Config initialized successfully.
[2025-08-26 09:15:27,364][cesnet_dataset][INFO] - Configuration has been changed successfu

#### Custom

You can create your own custom transformer. It is recommended to derive from Transformer base class.

In [9]:
class CustomTransformer(Transformer):
    def __init__(self):
        super().__init__()
        
        self.max = None
        self.min = None
    
    def transform(self, data):
        return (data - self.min) / (self.max - self.min)
    
    def fit(self, data):
        self.partial_fit(data)
    
    def partial_fit(self, data):
        
        if self.max is None and self.min is None:
            self.max = np.max(data, axis=0)
            self.min = np.min(data, axis=0)
            return
        
        temp_max = np.max(data, axis=0)
        temp = np.vstack((self.max, temp_max)) 
        self.max = np.max(temp, axis=0)
        
        temp_min = np.min(data, axis=0)
        temp = np.vstack((self.min, temp_min)) 
        self.min = np.min(temp, axis=0)            

In [10]:
config = DisjointTimeBasedConfig(train_ts=500, val_ts=None, test_ts=None, train_time_period=0.5, features_to_take=["n_flows", "n_packets"],
                           transform_with=CustomTransformer, nan_threshold=0.5, random_state=1500)
disjoint_dataset.set_dataset_config_and_initialize(config, display_config_details=True, workers=0)

[2025-08-26 09:15:27,375][disjoint_time_based_config][INFO] - Quick validation succeeded.
[2025-08-26 09:15:27,411][disjoint_time_based_config][INFO] - Finalization and validation completed successfully.
[2025-08-26 09:15:27,414][cesnet_dataset][INFO] - Updating config for train set.
100%|██████████| 500/500 [00:00<00:00, 1072.06it/s]
[2025-08-26 09:15:27,882][cesnet_dataset][INFO] - Config initialized successfully.



Config Details
    Used for database: CESNET-TimeSeries24
    Aggregation: AgreggationType.AGG_10_MINUTES
    Source: SourceType.IP_ADDRESSES_SAMPLE

    Time series
        Train time series IDs: [182151  10158  65072  10196 338309 ... 175742 659213  11188  73422 483796], Length=60
        Val time series IDs: None
        Test time series IDs: None
    Time periods
        Train time periods: range(0, 20149)
        Val time periods: None
        Test time periods: None
    Features
        Taken features: ['n_flows', 'n_packets']
        Default values: [0. 0.]
        Time series ID included: True
        Time included: True    
        Time format: TimeFormat.ID_TIME
    Sliding window
        Sliding window size: None
        Sliding window prediction size: None
        Sliding window step size: 1
    Fillers
        Filler type: None
    Transformers
        Transformer type: CustomTransformer (Custom)
        Are transformers premade: False
        Are premade transformers par

In [11]:
disjoint_dataset.get_train_df(workers=0).head(10)

,id_ip,id_time,n_flows,n_packets
0,182151.0,0.0,0.000079,2.240487e-07
1,182151.0,1.0,0.000045,1.280278e-07
2,182151.0,2.0,0.000045,1.280278e-07
3,182151.0,3.0,0.000056,1.600348e-07
4,182151.0,4.0,0.000090,2.560557e-07
5,182151.0,5.0,0.000034,9.602087e-08
6,182151.0,6.0,0.000068,1.920417e-07
7,182151.0,7.0,0.000045,1.280278e-07
8,182151.0,8.0,0.000102,3.840835e-07
9,182151.0,9.0,0.000000,0.000000e+00


In [12]:
disjoint_dataset.get_transformers()

Or later with:

In [13]:
disjoint_dataset.update_dataset_config_and_initialize(transform_with=CustomTransformer, partial_fit_initialized_transformers="config", workers=0)
# Or
disjoint_dataset.apply_transformer(transform_with=CustomTransformer, partial_fit_initialized_transformers="config", workers=0)

[2025-08-26 09:15:28,094][cesnet_dataset][INFO] - Re-initialization is required.
[2025-08-26 09:15:28,129][disjoint_time_based_config][INFO] - Finalization and validation completed successfully.
[2025-08-26 09:15:28,132][cesnet_dataset][INFO] - Updating config for train set.
100%|██████████| 60/60 [00:00<00:00, 420.35it/s]
[2025-08-26 09:15:28,277][cesnet_dataset][INFO] - Config initialized successfully.
[2025-08-26 09:15:28,277][cesnet_dataset][INFO] - Configuration has been changed successfuly.
[2025-08-26 09:15:28,278][cesnet_dataset][INFO] - Re-initialization is required.
[2025-08-26 09:15:28,365][disjoint_time_based_config][INFO] - Finalization and validation completed successfully.
[2025-08-26 09:15:28,369][cesnet_dataset][INFO] - Updating config for train set.
100%|██████████| 60/60 [00:00<00:00, 424.77it/s]
[2025-08-26 09:15:28,511][cesnet_dataset][INFO] - Config initialized successfully.
[2025-08-26 09:15:28,512][cesnet_dataset][INFO] - Configuration has been changed successfu

#### Using already fitted transformer

- When `partial_fit_initialized_transformer` is False (default value), transformer has no requirement for `partial_fit` nor for train set.

In [14]:
config = DisjointTimeBasedConfig(train_ts=500, val_ts=None, test_ts=None, train_time_period=0.5, features_to_take=["n_flows", "n_packets"],
                           transform_with=CustomTransformer, nan_threshold=0.5, random_state=1500)
disjoint_dataset.set_dataset_config_and_initialize(config, display_config_details=False, workers=0)

fitted_transformer = disjoint_dataset.get_transformers()

[2025-08-26 09:15:28,516][disjoint_time_based_config][INFO] - Quick validation succeeded.
[2025-08-26 09:15:28,552][disjoint_time_based_config][INFO] - Finalization and validation completed successfully.
[2025-08-26 09:15:28,555][cesnet_dataset][INFO] - Updating config for train set.
100%|██████████| 500/500 [00:00<00:00, 1127.71it/s]
[2025-08-26 09:15:29,001][cesnet_dataset][INFO] - Config initialized successfully.


In [15]:
config = DisjointTimeBasedConfig(train_ts=500, val_ts=500, test_ts=None, train_time_period=0.5, val_time_period=0.5, features_to_take=["n_flows", "n_packets"],
                           transform_with=fitted_transformer, nan_threshold=0.5, random_state=999)
disjoint_dataset.set_dataset_config_and_initialize(config, display_config_details=True, workers=0)

[2025-08-26 09:15:29,005][disjoint_time_based_config][INFO] - Quick validation succeeded.
[2025-08-26 09:15:29,072][disjoint_time_based_config][INFO] - Finalization and validation completed successfully.
[2025-08-26 09:15:29,075][cesnet_dataset][INFO] - Updating config for train set.
100%|██████████| 500/500 [00:00<00:00, 1090.67it/s]
[2025-08-26 09:15:29,537][cesnet_dataset][INFO] - Updating config for val set.
100%|██████████| 500/500 [00:00<00:00, 971.88it/s] 
[2025-08-26 09:15:30,053][cesnet_dataset][INFO] - Config initialized successfully.



Config Details
    Used for database: CESNET-TimeSeries24
    Aggregation: AgreggationType.AGG_10_MINUTES
    Source: SourceType.IP_ADDRESSES_SAMPLE

    Time series
        Train time series IDs: [  4380  35210 322201    190 307400 ...  29194 617662 677144 211973 612051], Length=65
        Val time series IDs: [ 272278  450900 1672430  252575   67484 ...   1672 471658 479584   1370 175742], Length=75
        Test time series IDs: None
    Time periods
        Train time periods: range(0, 20149)
        Val time periods: range(20149, 40298)
        Test time periods: None
    Features
        Taken features: ['n_flows', 'n_packets']
        Default values: [0. 0.]
        Time series ID included: True
        Time included: True    
        Time format: TimeFormat.ID_TIME
    Sliding window
        Sliding window size: None
        Sliding window prediction size: None
        Sliding window step size: 1
    Fillers
        Filler type: None
    Transformers
        Transformer type: C

In [16]:
disjoint_dataset.get_val_df(workers=0).head(10)

,id_ip,id_time,n_flows,n_packets
0,272278.0,20149.0,0.0,0.0
1,272278.0,20150.0,0.0,0.0
2,272278.0,20151.0,0.0,0.0
3,272278.0,20152.0,0.0,0.0
4,272278.0,20153.0,0.0,0.0
5,272278.0,20154.0,0.0,0.0
6,272278.0,20155.0,0.0,0.0
7,272278.0,20156.0,0.0,0.0
8,272278.0,20157.0,0.0,0.0
9,272278.0,20158.0,0.0,0.0


Below you can see how transformer works even without train set.

In [17]:
config = DisjointTimeBasedConfig(train_ts=None, val_ts=500, test_ts=None, val_time_period=0.5, features_to_take=["n_flows", "n_packets"],
                           transform_with=fitted_transformer, nan_threshold=0.5, random_state=999)
disjoint_dataset.set_dataset_config_and_initialize(config, display_config_details=True, workers=0)

[2025-08-26 09:15:30,349][disjoint_time_based_config][INFO] - Quick validation succeeded.
[2025-08-26 09:15:30,384][disjoint_time_based_config][INFO] - Finalization and validation completed successfully.
[2025-08-26 09:15:30,388][cesnet_dataset][INFO] - Updating config for val set.
100%|██████████| 500/500 [00:00<00:00, 1172.82it/s]
[2025-08-26 09:15:30,816][cesnet_dataset][INFO] - Config initialized successfully.



Config Details
    Used for database: CESNET-TimeSeries24
    Aggregation: AgreggationType.AGG_10_MINUTES
    Source: SourceType.IP_ADDRESSES_SAMPLE

    Time series
        Train time series IDs: None
        Val time series IDs: [  4380  35210 322201    190 307400 ...  29194 617662 677144 211973 612051], Length=65
        Test time series IDs: None
    Time periods
        Train time periods: None
        Val time periods: range(0, 20149)
        Test time periods: None
    Features
        Taken features: ['n_flows', 'n_packets']
        Default values: [0. 0.]
        Time series ID included: True
        Time included: True    
        Time format: TimeFormat.ID_TIME
    Sliding window
        Sliding window size: None
        Sliding window prediction size: None
        Sliding window step size: 1
    Fillers
        Filler type: None
    Transformers
        Transformer type: CustomTransformer (Custom)
        Are transformers premade: True
        Are premade transformers part

In [18]:
disjoint_dataset.get_val_df(workers=0).head(10)

,id_ip,id_time,n_flows,n_packets
0,4380.0,0.0,0.001253,0.000005
1,4380.0,1.0,0.001162,0.000004
2,4380.0,2.0,0.000756,0.000003
3,4380.0,3.0,0.000959,0.000004
4,4380.0,4.0,0.001219,0.000005
5,4380.0,5.0,0.001275,0.000005
6,4380.0,6.0,0.001456,0.000006
7,4380.0,7.0,0.001196,0.000004
8,4380.0,8.0,0.001433,0.000006
9,4380.0,9.0,0.001072,0.000004


##### Partial fitting on train set

Makes already fitted transformer to be fitted on new train set too. Must implement `partial_fit`.

In [19]:
config = DisjointTimeBasedConfig(train_ts=500, val_ts=500, test_ts=None, train_time_period=0.5, val_time_period=0.5, features_to_take=["n_flows", "n_packets"],
                           transform_with=fitted_transformer, partial_fit_initialized_transformer=True, nan_threshold=0.5, random_state=999)
disjoint_dataset.set_dataset_config_and_initialize(config, display_config_details=True, workers=0)

[2025-08-26 09:15:31,029][disjoint_time_based_config][INFO] - Quick validation succeeded.
[2025-08-26 09:15:31,096][disjoint_time_based_config][INFO] - Finalization and validation completed successfully.
[2025-08-26 09:15:31,099][cesnet_dataset][INFO] - Updating config for train set.
100%|██████████| 500/500 [00:00<00:00, 1120.59it/s]
[2025-08-26 09:15:31,550][cesnet_dataset][INFO] - Updating config for val set.
100%|██████████| 500/500 [00:00<00:00, 952.19it/s]
[2025-08-26 09:15:32,078][cesnet_dataset][INFO] - Config initialized successfully.



Config Details
    Used for database: CESNET-TimeSeries24
    Aggregation: AgreggationType.AGG_10_MINUTES
    Source: SourceType.IP_ADDRESSES_SAMPLE

    Time series
        Train time series IDs: [  4380  35210 322201    190 307400 ...  29194 617662 677144 211973 612051], Length=65
        Val time series IDs: [ 406306   65072      20 1672430  151737 ... 483796 476751    118  67484 338309], Length=75
        Test time series IDs: None
    Time periods
        Train time periods: range(0, 20149)
        Val time periods: range(20149, 40298)
        Test time periods: None
    Features
        Taken features: ['n_flows', 'n_packets']
        Default values: [0. 0.]
        Time series ID included: True
        Time included: True    
        Time format: TimeFormat.ID_TIME
    Sliding window
        Sliding window size: None
        Sliding window prediction size: None
        Sliding window step size: 1
    Fillers
        Filler type: None
    Transformers
        Transformer type: C

In [20]:
disjoint_dataset.get_val_df(workers=0).head(10)

,id_ip,id_time,n_flows,n_packets
0,406306.0,20149.0,0.000045,1.280278e-07
1,406306.0,20150.0,0.000056,1.600348e-07
2,406306.0,20151.0,0.000034,9.602087e-08
3,406306.0,20152.0,0.000079,2.560557e-07
4,406306.0,20153.0,0.000023,6.401392e-08
5,406306.0,20154.0,0.000023,6.401392e-08
6,406306.0,20155.0,0.000056,2.240487e-07
7,406306.0,20156.0,0.000079,3.200696e-07
8,406306.0,20157.0,0.000023,6.401392e-08
9,406306.0,20158.0,0.000045,1.280278e-07
